In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from fbprophet import Prophet

# Remove parts of the original signal

In [2]:
df = pd.read_csv("../data/raw/synthetic_basic/synthetic_basic_012.csv")
df.head()

,Unnamed: 0,Power,POA,Tamb,Wind,Degradation_rate_per_yr,soiling
0,2015-01-01 00:00:00-05:00,-1.0,0.0,5.1,0.0,-0.01432,1
1,2015-01-01 00:01:00-05:00,-1.0,0.0,5.1,0.0,-0.01432,1
2,2015-01-01 00:02:00-05:00,-1.0,0.0,5.1,0.0,-0.01432,1
3,2015-01-01 00:03:00-05:00,-1.0,0.0,5.1,0.0,-0.01432,1
4,2015-01-01 00:04:00-05:00,-1.0,0.0,5.1,0.0,-0.01432,1


In [3]:
# Parse the timestamp
df = df.rename(columns={'Unnamed: 0':'time'})
df['time'] = pd.to_datetime(df['time'])
df['time'] = df['time'].dt.tz_localize(None)

# Create the new column indicating minute of the day
df['minute'] = df.time.dt.minute + 60 * df.time.dt.hour

In [4]:
df.tail()

,time,Power,POA,Tamb,Wind,Degradation_rate_per_yr,soiling,minute
2629435,2019-12-31 23:55:00,-1.0,0.0,5.2,0.0,-0.01432,1,1435
2629436,2019-12-31 23:56:00,-1.0,0.0,5.2,0.0,-0.01432,1,1436
2629437,2019-12-31 23:57:00,-1.0,0.0,5.2,0.0,-0.01432,1,1437
2629438,2019-12-31 23:58:00,-1.0,0.0,5.2,0.0,-0.01432,1,1438
2629439,2019-12-31 23:59:00,-1.0,0.0,5.2,0.0,-0.01432,1,1439


In [7]:
def find_overlap_day_no_clip(df):
    # Input: dataframe with Power and minute columns
    clip_power = df.Power.max()
    night_power = -1

    latest_clip = df[df.Power == clip_power].minute.max()
    earliest_clip = df[df.Power == clip_power].minute.min()

#if 720 in df[df.Power == night_power].minute:
#    Print("Polar night!")

    latest_night = df[(df.Power == night_power) & (df.minute <= 720)].minute.max()
    earliest_night = df[(df.Power == night_power) & (df.minute >= 720)].minute.min()

    overlap = max(earliest_clip - latest_night, 0) + max(earliest_night - latest_clip, 0)
    return  latest_night, earliest_clip, latest_clip, earliest_night, overlap

In [8]:
find_overlap_day_no_clip(df)

(473, 588, 883, 1009, 241)

In [7]:
outs = [];
for i in range(1, 50):
    if i < 10:
        name = f"../data/raw/synthetic_weather/synthetic_weather_00{i}.csv"
    else:
        name = f"../data/raw/synthetic_weather/synthetic_weather_0{i}.csv"
    print(name)
    df = pd.read_csv(name)
    # Parse the timestamp
    df = df.rename(columns={'Unnamed: 0':'time'})
    df['time'] = pd.to_datetime(df['time'])
    df['time'] = df['time'].dt.tz_localize(None)

    # Create the new column indicating minute of the day
    df['minute'] = df.time.dt.minute + 60 * df.time.dt.hour
    _,_,_,_,val = find_overlap_day_no_clip(df)
    print(val)
    outs.append(val)


../data/raw/synthetic_weather/synthetic_weather_001.csv
0
../data/raw/synthetic_weather/synthetic_weather_002.csv
0
../data/raw/synthetic_weather/synthetic_weather_003.csv
0
../data/raw/synthetic_weather/synthetic_weather_004.csv
0
../data/raw/synthetic_weather/synthetic_weather_005.csv
0
../data/raw/synthetic_weather/synthetic_weather_006.csv
0
../data/raw/synthetic_weather/synthetic_weather_007.csv
0
../data/raw/synthetic_weather/synthetic_weather_008.csv
0
../data/raw/synthetic_weather/synthetic_weather_009.csv
0
../data/raw/synthetic_weather/synthetic_weather_010.csv
0
../data/raw/synthetic_weather/synthetic_weather_011.csv
0
../data/raw/synthetic_weather/synthetic_weather_012.csv
0
../data/raw/synthetic_weather/synthetic_weather_013.csv
0
../data/raw/synthetic_weather/synthetic_weather_014.csv
0
../data/raw/synthetic_weather/synthetic_weather_015.csv
0
../data/raw/synthetic_weather/synthetic_weather_016.csv
0
../data/raw/synthetic_weather/synthetic_weather_017.csv


KeyboardInterrupt: 

In [ ]:
print(outs)